In [31]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split, cross_val_score

import statsmodels.api as sm

In [32]:
train = pd.read_csv('../../cleaned_datasets/train.csv')
test = pd.read_csv('../../cleaned_datasets/test.csv')

***
## Feature Engineering
***

In [33]:
# combining Train and Test for feature engineering
whole = pd.concat([train, test], sort = False)


# dropping price so no columns contain direct price information
whole.drop(columns = 'saleprice', inplace = True)



# removing dummies so that this is feasible

whole = whole.iloc[:,:whole.columns.get_loc('yr_sold') + 1]

display(whole.head())

    


,index,id,pid,ms_subclass,lot_frontage,lot_area,overall_qual,overall_cond,year_built,year_remod/add,...,garage_area,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,misc_val,mo_sold,yr_sold
0,0.0,109.0,533352170.0,60.0,57.480147,13517.0,6.0,8.0,1976.0,2005.0,...,475.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0,3.0,2010.0
1,1.0,544.0,531379050.0,60.0,43.000000,11492.0,7.0,5.0,1996.0,1997.0,...,559.0,0.0,74.0,0.0,0.0,0.0,0.0,0.0,4.0,2009.0
2,2.0,153.0,535304180.0,20.0,68.000000,7922.0,5.0,7.0,1953.0,2007.0,...,246.0,0.0,52.0,0.0,0.0,0.0,0.0,0.0,1.0,2010.0
3,3.0,318.0,916386060.0,60.0,73.000000,9802.0,5.0,5.0,2006.0,2007.0,...,400.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2010.0
4,4.0,255.0,906425045.0,50.0,82.000000,14235.0,6.0,8.0,1900.0,1993.0,...,484.0,0.0,59.0,0.0,0.0,0.0,0.0,0.0,3.0,2010.0


In [34]:
# making features and concating to whole

features = pd.DataFrame(columns = range(1))


for col in whole.columns:
    
    unused_cols = list(whole.columns)
    
    for sub_col in unused_cols:
        
        
       features[f'{col} * {sub_col}'] = whole[col] * whole[sub_col]
          
        
    unused_cols.remove(col)

# features.drop(columns = 'index', inplace= True)


whole = pd.concat([whole, features], axis= 1)



In [35]:
whole.drop(columns= ['index', 0], inplace= True)

whole

,id,pid,ms_subclass,lot_frontage,lot_area,overall_qual,overall_cond,year_built,year_remod/add,mas_vnr_area,...,yr_sold * garage_area,yr_sold * wood_deck_sf,yr_sold * open_porch_sf,yr_sold * enclosed_porch,yr_sold * 3ssn_porch,yr_sold * screen_porch,yr_sold * pool_area,yr_sold * misc_val,yr_sold * mo_sold,yr_sold * yr_sold
0,109.0,533352170.0,60.0,57.480147,13517.0,6.0,8.0,1976.0,2005.0,289.0,...,954750.0,0.0,88440.0,0.0,0.0,0.0,0.0,0.0,6030.0,4040100.0
1,544.0,531379050.0,60.0,43.000000,11492.0,7.0,5.0,1996.0,1997.0,132.0,...,1123031.0,0.0,148666.0,0.0,0.0,0.0,0.0,0.0,8036.0,4036081.0
2,153.0,535304180.0,20.0,68.000000,7922.0,5.0,7.0,1953.0,2007.0,0.0,...,494460.0,0.0,104520.0,0.0,0.0,0.0,0.0,0.0,2010.0,4040100.0
3,318.0,916386060.0,60.0,73.000000,9802.0,5.0,5.0,2006.0,2007.0,0.0,...,804000.0,201000.0,0.0,0.0,0.0,0.0,0.0,0.0,8040.0,4040100.0
4,255.0,906425045.0,50.0,82.000000,14235.0,6.0,8.0,1900.0,1993.0,0.0,...,972840.0,0.0,118590.0,0.0,0.0,0.0,0.0,0.0,6030.0,4040100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,1662.0,527377110.0,60.0,80.000000,8000.0,6.0,6.0,1974.0,1974.0,0.0,...,979416.0,0.0,192672.0,0.0,0.0,0.0,0.0,0.0,22077.0,4028049.0
874,1234.0,535126140.0,60.0,90.000000,14670.0,6.0,7.0,1966.0,1999.0,410.0,...,963840.0,0.0,461840.0,0.0,0.0,0.0,0.0,0.0,16064.0,4032064.0
875,1373.0,904100040.0,20.0,55.000000,8250.0,5.0,5.0,1968.0,1968.0,0.0,...,646576.0,0.0,126504.0,0.0,0.0,0.0,0.0,0.0,16064.0,4032064.0
876,1672.0,527425140.0,20.0,60.000000,9000.0,4.0,6.0,1971.0,1971.0,0.0,...,1059696.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10035.0,4028049.0


In [36]:


train_ = whole.iloc[:2051]
test_ = whole.iloc[2051:]




train_['saleprice'] = train['saleprice']
test_['saleprice'] = test['saleprice']




train = train_
test = test_






C:\Users\Zachary\.conda\envs\dsi\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Zachary\.conda\envs\dsi\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [37]:
X = train.drop(columns='saleprice')
y = train['saleprice']



In [38]:
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
high_p_vals = list(model.pvalues[model.pvalues >= .05].index)[2:]
X.drop(columns = high_p_vals, inplace = True)


X.drop(columns = 'const', inplace=True)

C:\Users\Zachary\.conda\envs\dsi\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\Zachary\.conda\envs\dsi\lib\site-packages\statsmodels\base\model.py:1294: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\Zachary\.conda\envs\dsi\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\Zachary\.conda\envs\dsi\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\Zachary\.conda\envs\dsi\lib\site-packages\scipy\stats\_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [39]:
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
high_p_vals = list(model.pvalues[model.pvalues >= .05].index)[2:]
X.drop(columns = high_p_vals, inplace = True)


X.drop(columns = 'const', inplace=True)

In [41]:
features = list(X.columns)

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)



In [43]:
lr = LinearRegression()

y_train_log = y_train.map(np.log)
y_test_log = y_test.map(np.log)



print(cross_val_score(lr, X_train, y_train_log, cv=5).mean()
)




lr.fit(X_train, y_train_log)

lr.score(X_train, y_train_log), lr.score(X_test, y_test_log)




0.8216710652411792


(0.8759720267374894, 0.2626512284837206)

In [44]:
lr.fit(X, y.map(np.log))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [45]:


testing = test.drop(columns = 'saleprice')

testing = testing[features]

display(testing)

output = pd.DataFrame({'Id': test['id']})


output['SalePrice'] = lr.predict(testing)


output['SalePrice'] = output['SalePrice'].map(np.exp)

output['Id'] = [int(val) for val in output['Id']]

display(output)

pd.DataFrame.to_csv(output, '../../submissions/feat_engineer_multi_p_val_drop.csv', index = False)

,id,year_built,index * mas_vnr_area,index * bsmt_unf_sf,index * 3ssn_porch,id * open_porch_sf,id * 3ssn_porch,id * mo_sold,lot_area * lot_area,lot_area * overall_cond,...,3ssn_porch * garage_yr_blt,3ssn_porch * screen_porch,3ssn_porch * pool_area,screen_porch * 3ssn_porch,pool_area * low_qual_fin_sf,pool_area * 3ssn_porch,misc_val * lot_area,misc_val * overall_qual,mo_sold * id,yr_sold * bedroom_abvgr
0,2658.0,1910.0,0.0,0.0,0.0,159480.0,0.0,10632.0,83576164.0,73136.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10632.0,8024.0
1,2718.0,1977.0,0.0,1967.0,0.0,0.0,0.0,21744.0,93354244.0,38648.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21744.0,12036.0
2,2414.0,2006.0,0.0,200.0,0.0,57936.0,0.0,21726.0,292546816.0,85520.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21726.0,6018.0
3,1989.0,1923.0,0.0,2904.0,0.0,0.0,0.0,13923.0,72590400.0,51120.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13923.0,4014.0
4,625.0,1963.0,988.0,3140.0,0.0,47500.0,0.0,4375.0,90250000.0,47500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4375.0,6027.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,1662.0,1974.0,0.0,0.0,0.0,159552.0,0.0,18282.0,64000000.0,48000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18282.0,8028.0
874,1234.0,1966.0,358340.0,462346.0,0.0,283820.0,0.0,9872.0,215208900.0,102690.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9872.0,8032.0
875,1373.0,1968.0,0.0,183750.0,0.0,86499.0,0.0,10984.0,68062500.0,41250.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10984.0,6024.0
876,1672.0,1971.0,0.0,217248.0,0.0,0.0,0.0,8360.0,81000000.0,54000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8360.0,6021.0


,Id,SalePrice
0,2658,149538.244055
1,2718,176245.445901
2,2414,228103.414981
3,1989,113336.474077
4,625,163370.086337
...,...,...
873,1662,189533.224961
874,1234,213831.892505
875,1373,125247.946814
876,1672,108658.157389
